In [83]:
import os
import pandas as pd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# Use "conda install -c conda-forge holidays"
from datetime import date 
import holidays

In [84]:
os.getcwd()

'C:\\Users\\vijay.agrawal\\Final Project'

Import the datasets from the current working directory:

In [85]:
df_products=pd.read_csv("Apperal datasets/product_dataset.csv")

In [86]:
df_transactions=pd.read_csv("Apperal datasets/transaction_dataset.csv")

In [87]:
df_stores = pd.read_csv("Apperal datasets/store_dataset.csv", na_values = [""]) #Vijay

In [88]:
# Women income data obtained from 
#https://wallethub.com/edu/best-and-worst-states-for-women/10728/
#https://statusofwomendata.org/explore-the-data/employment-and-earnings/#chapter
#https://statusofwomendata.org/explore-the-data/

df_income = pd.read_csv("h08b.csv")

# Data Cleansing - df_products

In [89]:
# Determine the names of the columns in the df_products dataset:
df_products.columns.tolist()

['DBSKU',
 'DEPARTMENT',
 'CLASS',
 'SUBCLASS',
 'DEPARTMENT_NAME',
 'CLASS_NAME',
 'SUBCLASS_NAME']

## Feature Reduction: Remove unneeded columns

In [90]:
# Department Name, Class Name and Subclass Name bear no significance in our analysis - drop these columns:
df_products=df_products.drop(['DEPARTMENT_NAME', 'CLASS_NAME', 'SUBCLASS_NAME'], axis=1)

In [91]:
# List the column names after dropping the columns in the preceding command:
df_products.columns.tolist()

['DBSKU', 'DEPARTMENT', 'CLASS', 'SUBCLASS']

## Remove rows with null and missing DBSKU

In [92]:
# Determine null or missing values in the dataset and treat these values - 
# as seen below, column DBSKU has 2110 null or missing values:
df_products.isna().sum()

DBSKU         2110
DEPARTMENT       0
CLASS            0
SUBCLASS         0
dtype: int64

In [93]:
# The following command wil drop the records that have missing values in DBSKU:
df_products = df_products.dropna(axis=0, subset=['DBSKU'])

In [94]:
# Check for null values now - as seen below, there are no null values now. 
df_products.isna().sum()

DBSKU         0
DEPARTMENT    0
CLASS         0
SUBCLASS      0
dtype: int64

## Ensure right datatypes for the features

In [95]:
# Determine the datatypes in this dataset:
df_products.dtypes

DBSKU         float64
DEPARTMENT      int64
CLASS           int64
SUBCLASS        int64
dtype: object

In [96]:
# Convert DBSKU (Product Serial Number) to an integer datatype:
df_products['DBSKU'] = df_products['DBSKU'].astype('int64')

In [97]:
# Ensure that the data type was changed to int64 on executing the preceding command:
df_products.dtypes

DBSKU         int64
DEPARTMENT    int64
CLASS         int64
SUBCLASS      int64
dtype: object

## Remove duplicate SKUs

Remove *duplicate* DBSKU: *df_transactions* table uses DBSKU to identify products uniquely (its department, class, subclass). Hence DBSKU needs to be unique in the *df_products* table.

In [98]:
# Determine and verify duplicate entries:
duplicateRowsDF = df_products[df_products.duplicated(['DBSKU'])]
duplicateRowsDF.sort_values(by=['DBSKU']).head(3)

,DBSKU,DEPARTMENT,CLASS,SUBCLASS
21259,100503,10,4,40
21543,105593,10,3,31
17321,105593,10,3,31


In [99]:
# Remove duplicate records from the df_products table:
#keep = first will retain the first duplicated row in the dataset and drop the remaining duplicated rows. 
df_products.drop_duplicates(subset =['DBSKU'], keep = 'first', inplace = True) 

## Feature Reduction: Combine dependent features into a single feature

Analyse features **DEPARTMENT, CLASS and SUBCLASS**:

In [100]:
df_products.groupby(['DEPARTMENT','CLASS', 'SUBCLASS']).size().reset_index().rename(
    columns={0:'count'}).sort_values(by=['DEPARTMENT', 'CLASS'], ascending=True).head(7)

,DEPARTMENT,CLASS,SUBCLASS,count
0,10,1,5,480
1,10,1,6,860
2,10,2,20,4159
3,10,2,21,2555
4,10,3,30,155
5,10,3,31,681
6,10,3,32,280


As seen, combination of *DEPARTMENT* and *SUBCLASS* denotes a product. Combine **DEPARTMENT** and **SUBCLASS** into one column PRODUCT_CLASS and drop the *CLASS* column:

In [101]:
df_products['PRODUCT_CLASS'] = df_products['DEPARTMENT'].astype(str) + '_' + df_products['SUBCLASS'].astype(str)
df_products.head(3)

,DBSKU,DEPARTMENT,CLASS,SUBCLASS,PRODUCT_CLASS
0,2182204,12,3,32,12_32
1,2860882,12,3,31,12_31
2,2695858,12,5,50,12_50


In [102]:
df_products['PRODUCT_CLASS'] = df_products['PRODUCT_CLASS'].astype('str')

In [103]:
# Drop Class and Subclass as we now have a combined feature called product_class:
df_products = df_products.drop(['CLASS', 'SUBCLASS'], axis=1)

In [104]:
df_products.head()

,DBSKU,DEPARTMENT,PRODUCT_CLASS
0,2182204,12,12_32
1,2860882,12,12_31
2,2695858,12,12_50
3,675793,10,10_41
4,2864173,12,12_40


In [105]:
# Export the cleaned file as df_products_cleaned.csv:
df_products.to_csv('df_products_clean.csv', index=False)

 # Data Cleansing - df_stores

In [106]:
# Determine the names of the columns in the df_stores dataset:
df_stores.columns.tolist()

['LOC_IDNT', 'CITY', 'STATE', 'STORE_TYPE', 'POSTAL_CD', 'STORE_SIZE']

*Postal Code* and *City* are redundant columns representing the same information. We will keep it for now to perform exploratory analysis and then remove it before doing modeling.

## Impute Null/missing values

In [107]:
# Determine null or missing values in the dataset and treat these values - as seen below, 
# column STORE_SIZE and STORE_TYPE have null or missing values:
df_stores.isna().sum()

LOC_IDNT        0
CITY            0
STATE           0
STORE_TYPE     33
POSTAL_CD       0
STORE_SIZE    184
dtype: int64

In [108]:
# Replace null values in the STORE_TYPE with value 'NOT APPLICABLE'
# There are certain rows in the dataset that already have this value - NOT APPLICABLE:
# df_stores['STORE_TYPE'].fillna('No location format', inplace=True) # Vijay
df_stores["STORE_TYPE"].fillna("NOT APPLICABLE", inplace = True) # Dhanush

In [109]:
# Replace the null values in the STORE_SIZE with the mean STORE_SIZE value:
df_stores = df_stores.replace(to_replace = np.nan, value =df_stores['STORE_SIZE'].mean())

In [110]:
# Check for non null columns - the following code should display the number of non null columns:
df_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 6 columns):
LOC_IDNT      1303 non-null int64
CITY          1303 non-null object
STATE         1303 non-null object
STORE_TYPE    1303 non-null object
POSTAL_CD     1303 non-null int64
STORE_SIZE    1303 non-null float64
dtypes: float64(1), int64(2), object(3)
memory usage: 61.2+ KB


## Remove invalid values

In [111]:
# Check for the unique state values in the dataset - we notice invalid information with STATE as -1 -
# this has to be removed:
df_stores['STATE'].unique()

array(['MO', 'NY', 'FL', 'CA', 'NH', 'MA', 'WV', 'IL', 'MD', 'NJ', 'AZ',
       'TX', 'IA', 'CT', 'KY', 'LA', 'TN', 'GA', 'OK', 'MI', 'WI', 'OH',
       'SC', 'KS', 'VA', 'PA', 'MN', 'DC', 'NV', 'IN', 'DE', 'AL', 'CO',
       'AR', 'NC', 'UT', 'RI', 'WY', 'MS', 'ND', 'WA', 'MT', 'ID', 'ME',
       'VT', 'OR', 'NM', 'NE', '-1', 'SD'], dtype=object)

In [112]:
# Remove the invalid state code:
df_stores = df_stores[df_stores['STATE'] != '-1']

In [113]:
# Convert CITY, STATE and STORE_TYPE into features of type string:
df_stores['CITY'] = df_stores['CITY'].astype('str')
df_stores['STATE'] = df_stores['STATE'].astype('str')
df_stores['STORE_TYPE'] = df_stores['STORE_TYPE'].astype('str')

In [114]:
# Determine the number of stores by type:
df_stores.groupby('STORE_TYPE')['LOC_IDNT'].nunique()

STORE_TYPE
Downtown Store           34
Freestanding Store        4
Lifestyle Center         12
Mega Outlet Mall         30
Mini Mall                15
NOT APPLICABLE           51
Outlet Mall              39
Outlet Strip            213
Power Strip             345
Regional Mall            62
Strip Store             480
Tourist Outlet Mall       2
Tourist Outlet Strip     15
Name: LOC_IDNT, dtype: int64

Based on the preceding information, most store types are either Strip or Malls - categorise store types based on this information to determine whether profits are driven by these values

## Reduce cardinality of a Store Type for better analysis

In [115]:
import re
r1 = re.compile(r'.*(Mall).*')
r2 = re.compile(r'.*(Strip).*')
r3 = re.compile(r'.*(Downtown).*|.*(Freestanding).*|.*(Lifestyle).*|.*(APPLICABLE).*')

In [116]:
# perform regex matching based on r1, r2 and r3 defined above to create 3 new columns - 
# one for each store type - MALLS, STRIPS and ALL OTHERS. 
df_stores['STORE_TYPE_MALL'] = df_stores.apply(lambda row: bool(r1.match(row['STORE_TYPE'])), axis = 1) 
df_stores['STORE_TYPE_STRIP'] = df_stores.apply(lambda row: bool(r2.match(row['STORE_TYPE'])), axis = 1) 
df_stores['STORE_TYPE_OTHER'] = df_stores.apply(lambda row: bool(r3.match(row['STORE_TYPE'])), axis = 1)

In [117]:
# Convert boolean values True and False into 1's and 0's:
df_stores['STORE_TYPE_MALL'] = df_stores['STORE_TYPE_MALL'].astype(int)
df_stores['STORE_TYPE_STRIP'] = df_stores['STORE_TYPE_STRIP'].astype(int)
df_stores['STORE_TYPE_OTHER'] = df_stores['STORE_TYPE_OTHER'].astype(int)

In [118]:
# Verify the new df_stores dataset:
df_stores.head(3)

,LOC_IDNT,CITY,STATE,STORE_TYPE,POSTAL_CD,STORE_SIZE,STORE_TYPE_MALL,STORE_TYPE_STRIP,STORE_TYPE_OTHER
0,249,ST LOUIS,MO,Strip Store,63119,3963.0,0,1,0
1,401,PATCHOGUE,NY,Power Strip,11772,3378.0,0,1,0
2,644,NAPLES,FL,Outlet Strip,34114,3652.0,0,1,0


In [119]:
# Drop the column STORE_TYPE as we now have 3 new columns based on STORE_TYPES:
df_stores = df_stores.drop(['STORE_TYPE'], axis=1)

In [120]:
# Data cleansing for STORE_SIZE:
df_stores.STORE_SIZE.describe()

count    1302.000000
mean     3345.776586
std       769.549478
min         1.000000
25%      3064.000000
50%      3345.776586
75%      3699.250000
max      6533.000000
Name: STORE_SIZE, dtype: float64

In [121]:
# Import TQDM to view progress bar:
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

Notice that the store size is *incorrect* with 1 sq ft (minimum value) - this has to corrected - wherever the store size is 1, replace with the *mean store size* of that **particular** *store type*:

## Impute invalid values (that dont make practical sense) in Store Size

In [122]:
for index, row in tqdm(df_stores.iterrows()):
    if ((row['STORE_SIZE'] == 1.0) and (row['STORE_TYPE_MALL'] == 1)):
        df_stores.at[index,'STORE_SIZE'] = df_stores[df_stores['STORE_TYPE_MALL'] == 1]['STORE_SIZE'].mean()
    elif ((row['STORE_SIZE'] == 1.0) and (row['STORE_TYPE_STRIP'] == 1)):
        df_stores.at[index,'STORE_SIZE'] = df_stores[df_stores['STORE_TYPE_STRIP'] == 1]['STORE_SIZE'].mean()
    elif ((row['STORE_SIZE'] == 1.0) and (row['STORE_TYPE_OTHER'] == 1)):
        df_stores.at[index,'STORE_SIZE'] = df_stores[df_stores['STORE_TYPE_OTHER'] == 1]['STORE_SIZE'].mean()

In [123]:
# Round off store sizes to the nearest hundred and remove any decimals:
df_stores['STORE_SIZE'] = np.ceil(df_stores['STORE_SIZE']/100)*100
df_stores['STORE_SIZE'] = df_stores['STORE_SIZE'].astype(int)

In [124]:
# The unique store sizes are as follows:
df_stores['STORE_SIZE'].unique()

array([4000, 3400, 3700, 2600, 3100, 2900, 2300, 4800, 4900, 3200, 3300,
       3500, 5200, 2800, 3600, 4700, 3800, 3900, 4300, 4400, 4500, 2500,
       4100, 5000, 2700, 3000, 4200, 6100, 4600, 5100, 5300, 5700, 5600,
       2400, 2100, 5400, 1900, 2200, 6600, 6300,  100, 6000, 1800],
      dtype=int64)

## DATA AUGMENTATION - enahnce dataset by adding Income and Gender Data

**MERGE** - *df_stores* and *df_income* - **DF_INCOME** is an external dataset to enhance information - this supplies information about *women employment percentage* by STATE - the value fashion retailer sells clothing for working women. 

In [125]:
# Merge these 2 datasets based on common column - STATE: 
df_stores = pd.merge(df_stores, df_income, on='STATE')

In [126]:
# Verify that the datasets have been merged:
df_stores.head(3)

,LOC_IDNT,CITY,STATE,POSTAL_CD,STORE_SIZE,STORE_TYPE_MALL,STORE_TYPE_STRIP,STORE_TYPE_OTHER,STATE_NAME,3YR_MEDIAN_HOUSEHLD_INCOME,PCNT_WOMEN_EMPLOYED,PCNT_WOMEN_EMPLOYED_RANK
0,249,ST LOUIS,MO,63119,4000,0,1,0,Missouri,58111,59.9%,22
1,925,KANSAS CITY,MO,64153,3400,0,0,1,Missouri,58111,59.9%,22
2,39,ST JOSEPH,MO,64506,3900,0,1,0,Missouri,58111,59.9%,22


In [127]:
df_stores = df_stores.rename(columns={"3YR_MEDIAN_HOUSEHLD_INCOME": "MEDIAN_HOUSEHLD_INCOME"})
df_stores['MEDIAN_HOUSEHLD_INCOME'].describe()

count     1302.000000
mean     61729.072965
std       7632.152664
min      42283.000000
25%      57367.000000
50%      61741.000000
75%      67891.000000
max      77525.000000
Name: MEDIAN_HOUSEHLD_INCOME, dtype: float64

In [128]:
# Bin 3 Year household median income (by state) based on the following cut points - 
# these cut points have been obtained from the minimum, 25th, 50th, 75th and maximum percentile values 
# of the field MEDIAN_HOUSEHLD_INCOME:
cut_points = [0, 42283, 57367, 61741, 67891, 77525]
labels = [1, 2, 3, 4, 5]
df_stores['MEDIAN_HOUSEHLD_INCOME_BINS'] = pd.cut(df_stores['MEDIAN_HOUSEHLD_INCOME'],
                                                      bins=cut_points,labels=labels,include_lowest=True)

In [129]:
# Convert percentage of women employed (by state) to float and round of 2 decimals. 
df_stores['PCNT_WOMEN_EMPLOYED'] = (df_stores['PCNT_WOMEN_EMPLOYED'].str.strip('%').astype('float64')/100).round(2)

In [130]:
# This is a redundant column and the information is already represented by PCNT_WOMEN_EMPLOYED - 
# hence this column can be dropped:
df_stores = df_stores.drop(['PCNT_WOMEN_EMPLOYED_RANK'], axis=1)

In [131]:
# Rename lOC_IDNT to LOC_INDT to merge with df_transactions table:
df_stores = df_stores.rename(columns={"LOC_IDNT": "LOC_INDT"})

In [132]:
# Sort LOC_INDT and create a sequential value for each LOC_INDT - now the values are scattered:
df_stores = df_stores.sort_values(by ='LOC_INDT')
df_stores.insert(1, 'LOC_INDT_SEQ', range(1, 1 + len(df_stores)))

In [133]:
# Verify the dataset:
df_stores.head()

,LOC_INDT,LOC_INDT_SEQ,CITY,STATE,POSTAL_CD,STORE_SIZE,STORE_TYPE_MALL,STORE_TYPE_STRIP,STORE_TYPE_OTHER,STATE_NAME,MEDIAN_HOUSEHLD_INCOME,PCNT_WOMEN_EMPLOYED,MEDIAN_HOUSEHLD_INCOME_BINS
720,2,1,NORWALK,CT,6851,6100,0,0,1,Connecticut,75251,0.63,5
682,3,2,MCALLEN,TX,78504,3600,0,1,0,Texas,59041,0.58,3
987,4,3,VIRGINIA BEACH,VA,23456,3700,0,1,0,Virginia,67597,0.61,4
164,5,4,NANUET,NY,10954,2600,0,1,0,New York,61741,0.59,3
1084,6,5,LAS VEGAS,NV,89106,3300,0,1,0,Nevada,55662,0.59,2


  Lets cleanup store_size

In [134]:
df_stores.STORE_SIZE.describe()

count    1302.000000
mean     3470.046083
std       581.307465
min       100.000000
25%      3200.000000
50%      3400.000000
75%      3700.000000
max      6600.000000
Name: STORE_SIZE, dtype: float64

  As can be seen, > 50% stores are in range of 3000-3700 sq ft
  Some stores have '1' as size, which is clearly wrong - we need to impute/drop them

In [135]:
df_stores.STORE_SIZE.value_counts().head()

3400    264
3300     93
3200     80
3500     77
3700     75
Name: STORE_SIZE, dtype: int64

In [136]:
#If any store has size of 1, impute it
df_stores['STORE_SIZE'] = df_stores['STORE_SIZE'].apply(lambda x:df_stores['STORE_SIZE'].mean() if x == 1.0 else x)

In [137]:
# Export the dataset as df_stores_clean.csv:
df_stores.to_csv('df_stores_clean.csv', index=False)

 # Data Cleansing - df_transactions

In [138]:
# Randomly choose 55% of the data from the df_transactions for processing - 
# this is due to save computing power as df_transactions has close to 10.3 million records:
# TODO For final submission, comment this out so that full dataset is taken
#df_transactions = df_transactions.sample(frac =.55, random_state = 100)

In [139]:
# we will be processing close to 7 million records as opposed to 10.3 million records:
df_transactions.shape

(13053149, 9)

In [140]:
# Determine the columns in this dataset df_transactions:
df_transactions.columns.tolist()

['DAY_DT',
 'LOC_INDT',
 'DBSKU',
 'ONLINE_FLAG',
 'FULL_PRICE_IND',
 'TOTAL_SALES',
 'TOTAL_UNITS',
 'TOTAL_SALES_PRFT',
 'TOTAL_COST']

## Drop transactions that dont have SKU entered

In [141]:
# Check for null values in this dataset df_transactions:
df_transactions.isna().sum()

DAY_DT                0
LOC_INDT              0
DBSKU               804
ONLINE_FLAG           0
FULL_PRICE_IND        0
TOTAL_SALES           0
TOTAL_UNITS           0
TOTAL_SALES_PRFT      0
TOTAL_COST            0
dtype: int64

Notice that there are 429 records (in DBSKU) that have null or missing values in this dataset - given that this is a very neglible number compared to the number of records in the dataset, we will be dropping the records. 

In [142]:
# drop records that have null or missing values:
df_transactions = df_transactions.dropna(axis=0, subset=['DBSKU'])

In [143]:
# Verify that the null records have been dropped:
df_transactions.isna().sum()

DAY_DT              0
LOC_INDT            0
DBSKU               0
ONLINE_FLAG         0
FULL_PRICE_IND      0
TOTAL_SALES         0
TOTAL_UNITS         0
TOTAL_SALES_PRFT    0
TOTAL_COST          0
dtype: int64

## Ensure proper datatypes of DBSKU

In [144]:
# Convert DBSKU to type int64 - since this feature was converted to type int in df_products.
df_transactions['DBSKU'] = df_transactions['DBSKU'].astype('int64')

## Data Augmentation. Add a "Holiday" column to analyze if Sales and Profits are impacted on holidays

Determine whether the date (DAY_DT) falls on a holiday - will sales on a holiday have an impact on selling price (and profit)? We will be generating a column called HOLIDAYS that will hold this information for every DAY_DT in the table *df_transactions*:

In [145]:
usa_holidays = holidays.UnitedStates()

In [146]:
df_transactions_date_list = df_transactions['DAY_DT'].tolist()
df_transactions_date_boolean = []
for i in tqdm(df_transactions_date_list):
        df_transactions_date_boolean.append(i in usa_holidays)

In [147]:
# Append list which has the holiday information to the dataframe df_transactions:
df_transactions['HOLIDAY'] = df_transactions_date_boolean

In [148]:
# Verify holiday mapping:
df_transactions[df_transactions['HOLIDAY'] == True].tail(5)

,DAY_DT,LOC_INDT,DBSKU,ONLINE_FLAG,FULL_PRICE_IND,TOTAL_SALES,TOTAL_UNITS,TOTAL_SALES_PRFT,TOTAL_COST,HOLIDAY
13052929,2018-05-28,753,672766,0,NFP,25.49,1.0,5.99,19.5,True
13052983,2018-07-04,704,718601,0,FP,54.00,1.0,37.00,17.0,True
13053028,2018-05-28,1330,682609,0,NFP,47.26,1.0,25.26,22.0,True
13053029,2018-05-28,932,672568,0,NFP,18.81,1.0,5.81,13.0,True
13053081,2018-05-28,1333,2178673,0,NFP,47.44,1.0,30.44,17.0,True


In [149]:
# Convert True and False mappings to 1's and 0's:
df_transactions['HOLIDAY'] = df_transactions['HOLIDAY'].astype('int64')

In [150]:
# Convert DAY_DT to a DATE datatype. Is is an object data type at the moment. 
df_transactions['DAY_DT'].head(3)

0    2015-09-26
1    2015-08-02
2    2015-10-21
Name: DAY_DT, dtype: object

In [151]:
# Convert to DATE data type:
df_transactions['DAY_DT']=pd.to_datetime(df_transactions['DAY_DT'])

In [152]:
# Now using the DAY_DT column, extract the month of the transaction:
df_transactions['MONTH'] = pd.DatetimeIndex(df_transactions['DAY_DT']).month

## Data Augmentation - add "Season" column to enable analysis of impact of Season on Sales/Profit - given it is womens apparel business

In [ ]:
# Determine the SEASON based on the month column:
# 1 = Winter (Months 12, 1, 2)
# 2 = Spring (Months 3, 4, 5)
# 3 = Summer (Months 6, 7, 8)
# 4 = Fall (Months 9, 10, 11)
# The logic below takes almost 30mins to run. Consider using lambda function to reduce execution time
for index, row in tqdm(df_transactions.iterrows()):
    if   ((row['MONTH'] == 12) or (row['MONTH'] == 1) or (row['MONTH'] == 2)):
        df_transactions.at[index,'SEASON'] = '1' # Winter
    elif ((row['MONTH'] == 3) or (row['MONTH'] == 4) or (row['MONTH'] == 5)):
        df_transactions.at[index,'SEASON'] = '2' # Spring
    elif ((row['MONTH'] == 6) or (row['MONTH'] == 7) or (row['MONTH'] == 8)):
        df_transactions.at[index,'SEASON'] = '3' # Summer
    else:
        df_transactions.at[index,'SEASON'] = '4' # Fall

In [72]:
# Verify these new columns:
df_transactions.head(3)

,DAY_DT,LOC_INDT,DBSKU,ONLINE_FLAG,FULL_PRICE_IND,TOTAL_SALES,TOTAL_UNITS,TOTAL_SALES_PRFT,TOTAL_COST,HOLIDAY,MONTH,SEASON
8159878,2017-06-14,520,626846,0,FP,44.0,1.0,29.25,14.75,0,6,3


## Unit Sale Price, Cost Price and Profit Per Unit will be used for analysis. Add those columns

In [73]:
# Determine UNIT_COST_PRICE = TOTAL COST PRICE OF SALE DIVIDED BY TOTAL UNITS SOLD:
df_transactions['UNIT_COST_PRICE'] = (df_transactions['TOTAL_COST']/df_transactions['TOTAL_UNITS']).round(2)

In [74]:
# Determine UNIT_SELLING_PRICE = TOTAL SELLING PRICE OF SALE DIVIDED BY TOTAL UNITS SOLD:
df_transactions['UNIT_SELLING_PRICE'] = (df_transactions['TOTAL_SALES']/df_transactions['TOTAL_UNITS']).round(2)

In [75]:
# Determine PROFIT_PER_UNIT = UNIT SELLING PRICE - UNIT COST PRICE:
df_transactions['PROFIT_PER_UNIT'] = (df_transactions['UNIT_SELLING_PRICE'] - df_transactions['UNIT_COST_PRICE']).round(2)

In [76]:
# For better clarity rename column TOTAL UNITS to TOTAL_UNITS_SOLD:
df_transactions = df_transactions.rename(columns={"TOTAL_UNITS": "TOTAL_UNITS_SOLD"})

In [77]:
# Column FULL_PRICE_INDC has 2 values - NFP and FP - 
# determines whether goods were sold at full price or not - map these NFP and FP values as 0's and 1's respectively:
df_transactions['FULL_PRICE_INDC'] = df_transactions['FULL_PRICE_IND'].apply(lambda x: 0 if x == 'NFP' else 1)

In [78]:
# Drop FULL_PRICE_IND and rename the new column FULL_PRICE_INDC as FULL_PRIC_IND
df_transactions = df_transactions.drop(['FULL_PRICE_IND'], axis=1)
df_transactions = df_transactions.rename(columns={"FULL_PRICE_INDC": "FULL_PRICE_IND"})

## Sales and Units Sold cannot be < 0. Drop rows with invalid values

In [79]:
# Modeling will be carried out only for transactions where TOTAL_SALES or TOTAL_UNITS_SOLD or
# TOTAL_COST is greater than 0 - so remove records where that is not the case:
df_transactions.drop(df_transactions[(df_transactions['TOTAL_SALES']<=0.0) |
                                     (df_transactions['TOTAL_UNITS_SOLD']<=0.0) | 
                                     (df_transactions['TOTAL_COST'] <= 0.0)].index , inplace=True)

In [80]:
df_transactions['FULL_PRICE_IND'].value_counts()

1    1
Name: FULL_PRICE_IND, dtype: int64

In [81]:
df_transactions['FULL_PRICE_IND'].dtype

dtype('int64')

  FULL_PRICE_IND is of correct type int64

## Save the cleaned up transactions data

In [82]:
# Export the dataset as df_transactions_clean.csv:
df_transactions.to_csv('df_transactions_full_clean.csv', index=False)